In [2]:
# dependencies
import pandas as pd
import os
from sqlalchemy import create_engine

In [3]:
# define filename
house_prices_file = "all_perth_310121.csv"

# assign filepath
house_prices_path = os.path.join("resources", house_prices_file)

In [4]:
# load in data
house_prices_df = pd.read_csv(house_prices_path)

# inspect data
house_prices_df.head()

,ADDRESS,SUBURB,PRICE,BEDROOMS,BATHROOMS,GARAGE,LAND_AREA,FLOOR_AREA,BUILD_YEAR,CBD_DIST,NEAREST_STN,NEAREST_STN_DIST,DATE_SOLD,POSTCODE,LATITUDE,LONGITUDE,NEAREST_SCH,NEAREST_SCH_DIST,NEAREST_SCH_RANK
0,1 Acorn Place,South Lake,565000,4,2,2.0,600,160,2003.0,18300,Cockburn Central Station,1800,09-2018\r,6164,-32.115900,115.842450,LAKELAND SENIOR HIGH SCHOOL,0.828339,NaN
1,1 Addis Way,Wandi,365000,3,2,2.0,351,139,2013.0,26900,Kwinana Station,4900,02-2019\r,6167,-32.193470,115.859553,ATWELL COLLEGE,5.524324,129.0
2,1 Ainsley Court,Camillo,287000,3,1,1.0,719,86,1979.0,22600,Challis Station,1900,06-2015\r,6111,-32.120578,115.993579,KELMSCOTT SENIOR HIGH SCHOOL,1.649178,113.0
3,1 Albert Street,Bellevue,255000,2,1,2.0,651,59,1953.0,17900,Midland Station,3600,07-2018\r,6056,-31.900547,116.038009,SWAN VIEW SENIOR HIGH SCHOOL,1.571401,NaN
4,1 Aman Place,Lockridge,325000,4,1,2.0,466,131,1998.0,11200,Bassendean Station,2000,11-2016\r,6054,-31.885790,115.947780,KIARA COLLEGE,1.514922,NaN


In [5]:
# inspect data types
house_prices_df.dtypes

ADDRESS              object
SUBURB               object
PRICE                 int64
BEDROOMS              int64
BATHROOMS             int64
GARAGE              float64
LAND_AREA             int64
FLOOR_AREA            int64
BUILD_YEAR          float64
CBD_DIST              int64
NEAREST_STN          object
NEAREST_STN_DIST      int64
DATE_SOLD            object
POSTCODE              int64
LATITUDE            float64
LONGITUDE           float64
NEAREST_SCH          object
NEAREST_SCH_DIST    float64
NEAREST_SCH_RANK    float64
dtype: object

In [6]:
# transform datatype to date
house_prices_df["DATE_SOLD"] = pd.to_datetime(house_prices_df["DATE_SOLD"])

# transform date to unix
house_prices_df["DATE_SOLD_UNIX"] = (house_prices_df["DATE_SOLD"] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

# confirm dates are unix format
house_prices_df["DATE_SOLD_UNIX"]

0        1535760000
1        1548979200
2        1433116800
3        1530403200
4        1477958400
            ...    
33651    1456790400
33652    1485907200
33653    1485907200
33654    1475280000
33655    1462060800
Name: DATE_SOLD_UNIX, Length: 33656, dtype: int64

In [16]:
# prepare the dataframe for the database export
all_house_prices_df = house_prices_df[['PRICE', 'DATE_SOLD_UNIX','ADDRESS','SUBURB','BEDROOMS','BATHROOMS','GARAGE','LAND_AREA','FLOOR_AREA','BUILD_YEAR','POSTCODE','LATITUDE','LONGITUDE','CBD_DIST','NEAREST_STN','NEAREST_STN_DIST','NEAREST_SCH','NEAREST_SCH_DIST','NEAREST_SCH_RANK']]
all_house_prices_df = all_house_prices_df.rename(columns={'PRICE':'price','DATE_SOLD_UNIX':'date','ADDRESS':'address','SUBURB':'suburb','BEDROOMS':'bedrooms','BATHROOMS':'bathrooms','GARAGE':'garage','LAND_AREA':'land_area','FLOOR_AREA':'floor_area','BUILD_YEAR':'build_year','POSTCODE':'postcode','LATITUDE':'latitude','LONGITUDE':'longitude','CBD_DIST':'cbd_dist', 'NEAREST_STN':'nearest_stn','NEAREST_STN_DIST':'nearest_stn_dist','NEAREST_SCH':'nearest_sch','NEAREST_SCH_DIST':'nearest_sch_dist','NEAREST_SCH_RANK':'nearest_sch_rank'})
all_house_prices_df['id'] = all_house_prices_df.index + 1

# check dataframe 
all_house_prices_df.head()

,price,date,address,suburb,bedrooms,bathrooms,garage,land_area,floor_area,build_year,postcode,latitude,longitude,cbd_dist,nearest_stn,nearest_stn_dist,nearest_sch,nearest_sch_dist,nearest_sch_rank,id
0,565000,1535760000,1 Acorn Place,South Lake,4,2,2.0,600,160,2003.0,6164,-32.115900,115.842450,18300,Cockburn Central Station,1800,LAKELAND SENIOR HIGH SCHOOL,0.828339,NaN,1
1,365000,1548979200,1 Addis Way,Wandi,3,2,2.0,351,139,2013.0,6167,-32.193470,115.859553,26900,Kwinana Station,4900,ATWELL COLLEGE,5.524324,129.0,2
2,287000,1433116800,1 Ainsley Court,Camillo,3,1,1.0,719,86,1979.0,6111,-32.120578,115.993579,22600,Challis Station,1900,KELMSCOTT SENIOR HIGH SCHOOL,1.649178,113.0,3
3,255000,1530403200,1 Albert Street,Bellevue,2,1,2.0,651,59,1953.0,6056,-31.900547,116.038009,17900,Midland Station,3600,SWAN VIEW SENIOR HIGH SCHOOL,1.571401,NaN,4
4,325000,1477958400,1 Aman Place,Lockridge,4,1,2.0,466,131,1998.0,6054,-31.885790,115.947780,11200,Bassendean Station,2000,KIARA COLLEGE,1.514922,NaN,5


In [17]:
# prepare the dataframe for the database export
cleaned_house_prices_df = all_house_prices_df[['id','price','date','address','suburb','bedrooms','bathrooms','land_area','latitude','longitude']]

# check dataframe
cleaned_house_prices_df.head()

,id,price,date,address,suburb,bedrooms,bathrooms,land_area,latitude,longitude
0,1,565000,1535760000,1 Acorn Place,South Lake,4,2,600,-32.115900,115.842450
1,2,365000,1548979200,1 Addis Way,Wandi,3,2,351,-32.193470,115.859553
2,3,287000,1433116800,1 Ainsley Court,Camillo,3,1,719,-32.120578,115.993579
3,4,255000,1530403200,1 Albert Street,Bellevue,2,1,651,-31.900547,116.038009
4,5,325000,1477958400,1 Aman Place,Lockridge,4,1,466,-31.885790,115.947780


In [18]:
# confirm datatypes
cleaned_house_prices_df.dtypes

id             int64
price          int64
date           int64
address       object
suburb        object
bedrooms       int64
bathrooms      int64
land_area      int64
latitude     float64
longitude    float64
dtype: object

In [19]:
# Create an engine for the sqlite database
engine = create_engine("sqlite:///db.sqlite", echo=False)

# Connect with database
connection = engine.connect()

In [20]:
# load sale info data
cleaned_house_prices_df.to_sql(name="house_prices", con=connection, if_exists='replace')

In [21]:
# Close database connection
connection.close()